In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
import feature_manager as fma
import rl.env_simple_crypto_trade as env
import importlib
import rl.models as rla
import config as cf
import numpy as np
from torch.nn import Tanh
import tr_utils

In [ ]:
importlib.reload(fma)

fm = fma.FeatureManager()
fm.import_data(symbol="BTCUSDT",timeframes=["1d","1w","1mo"])

fm.build_features(
    lags = [1,1,1],
    features=cf.FOR_1D_INDICATORS,
)

In [ ]:
importlib.reload(env)
importlib.reload(cf)

env_kwargs = cf.TRADE_ENV_PARAMETER
state_space = 10 + len(fm.cols)

full_env = env.CryptoTradingEnv(
    trade_timeframe="1d",
    df = fm.df, 
    state_space=state_space,
    indicators=fm.cols,
    **env_kwargs)


In [ ]:
importlib.reload(rla)
agent = rla.DRLTradeAgent(env=full_env)

catalog_name = "dqn_4layers_relu"

timestep = 960_000

selected_model = agent.load_model_from_checkpoint(
    "dqn",f"{catalog_name}/rl_model_{timestep}_steps")

In [ ]:
def func(x):
    return 1e-9
selected_model.set_env(full_env)
selected_model.lr_schedule = func
selected_model.learning_rate = 1e-9
selected_model.exploration_fraction= 0
selected_model.learning_starts=0
catalog_name = "dqn_4layers_relu_continue"

In [ ]:
selected_model.learn(
    total_timesteps = 5_000_000,
    tb_log_name = catalog_name,
    reset_num_timesteps = False,
    progress_bar = True,
)

In [ ]:
retrained_mode = agent.train_model(
    model = selected_model,
    total_timesteps = 5_000_000,
    checkpoint = True,
    catalog_name = catalog_name,
    save_frequency = 20_000,
    progress_bar = True
)

In [ ]:
agent.predict(
    model=selected_model,
    environment=full_env,
    render=False
)

In [ ]:
agent.describe_trades()

In [ ]:
agent.plot_results()

In [ ]:
agent.latest_model